# Named Entity Recognition

In [1]:
import nltk
import pandas as pd
import re

In [2]:
text = """
Bayern Munich, or FC Bayern, is a German sports club based in Munich, 
Bavaria, Germany. It is best known for its professional football team, 
which plays in the Bundesliga, the top tier of the German football 
league system, and is the most successful club in German football 
history, having won a record 26 national titles and 18 national cups. 
FC Bayern was founded in 1900 by eleven football players led by Franz John. 
Although Bayern won its first national championship in 1932, the club 
was not selected for the Bundesliga at its inception in 1963. The club 
had its period of greatest success in the middle of the 1970s when, 
under the captaincy of Franz Beckenbauer, it won the European Cup three 
times in a row (1974-76). Overall, Bayern has reached ten UEFA Champions 
League finals, most recently winning their fifth title in 2013 as part 
of a continental treble. 
"""

In [3]:
# text = """Thought-capable artificial beings appeared as storytelling devices in antiquity, and have been common in fiction, as in Mary Shelleys Frankenstein or Karel Capeks R.U.R. (Rossums Universal Robots). These characters and their fates raised many of the same issues now discussed in the ethics of artificial intelligence. The study of mechanical or formal reasoning began with philosophers and mathematicians in antiquity. The study of mathematical logic led directly to Alan Turings theory of computation, which suggested that a machine, by shuffling symbols as simple as 0 and 1, could simulate any conceivable act of mathematical deduction. This insight, that digital computers can simulate any process of formal reasoning, is known as the Church-Turing thesis. Along with concurrent discoveries in neurobiology, information theory and cybernetics, this led researchers to consider the possibility of building an electronic brain. Turing proposed that if a human could not distinguish between responses from a machine and a human, the machine could be considered intelligent. The first work that is now generally recognized as AI was McCullouch and Pitts 1943 formal design for Turing-complete artificial neurons. The field of AI research was born at a workshop at Dartmouth College in 1956. Attendees Allen Newell (CMU), Herbert Simon (CMU), John McCarthy (MIT), Marvin Minsky (MIT) and Arthur Samuel (IBM) became the founders and leaders of AI research. They and their students produced programs that the press described as astonishing: computers were learning checkers strategies (c. 1954) (and by 1959 were reportedly playing better than the average human),[32] solving word problems in algebra, proving logical theorems (Logic Theorist, first run c. 1956) and speaking English. By the middle of the 1960s, research in the U.S. was heavily funded by the Department of Defense and laboratories had been established around the world. AIs founders were optimistic about the future: Herbert Simon predicted, machines will be capable, within twenty years, of doing any work a man can do. Marvin Minsky agreed, writing, within a generation ... the problem of creating artificial intelligence will substantially be solved.  They failed to recognize the difficulty of some of the remaining tasks. Progress slowed and in 1974, in response to the criticism of Sir James Lighthill[36] and ongoing pressure from the US Congress to fund more productive projects, both the U.S. and British governments cut off exploratory research in AI. The next few years would later be called an AI winter, a period when obtaining funding for AI projects was difficult."""

In [4]:
def parse_document(document):
    document = re.sub('\n', ' ', document)
    if isinstance(document, str):
        document = document
    elif isinstance(document, unicode):
        return unicodedata.normalize('NFKD', document).encode('ascii', 'ignore')
    else:
        raise ValueError('Document is not string or unicode!')
    document = document.strip()
    sentences = nltk.sent_tokenize(document)
    sentences = [sentence.strip() for sentence in sentences]
    return sentences

In [5]:
sentences = parse_document(text)
tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]

In [6]:
tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
ne_chunked_sents = [nltk.ne_chunk(tagged) for tagged in tagged_sentences]

In [7]:
named_entities = []
for ne_tagged_sentence in ne_chunked_sents:
    for tagged_tree in ne_tagged_sentence:
        if hasattr(tagged_tree, "label"):
            entity_name = " ".join(c[0] for c in tagged_tree.leaves())
            entity_type = tagged_tree.label()
            named_entities.append((entity_name, entity_type))
            
named_entities = list(set(named_entities))
entity_frame = pd.DataFrame(named_entities,
                           columns=["Entity Name", "Entity Type"])
print(entity_frame)

          Entity Name   Entity Type
0             Bavaria           GPE
1              Bayern        PERSON
2            European  ORGANIZATION
3          Franz John        PERSON
4             Overall           GPE
5              Bayern           GPE
6          Bundesliga  ORGANIZATION
7                UEFA  ORGANIZATION
8              Munich  ORGANIZATION
9             Germany           GPE
10  Franz Beckenbauer        PERSON
11             Munich           GPE
12          FC Bayern  ORGANIZATION
13             German           GPE


In [8]:
# Stanford NER has to be downloaded
from nltk.tag import StanfordNERTagger
sn =StanfordNERTagger("/home/simon/nltk_data/stanford/stanford-ner-2018-02-27/classifiers/english.all.3class.distsim.crf.ser.gz",
                     path_to_jar="/home/simon/nltk_data/stanford/stanford-ner-2018-02-27/stanford-ner.jar")

In [9]:
ne_annotated_sentences = [sn.tag(sent) for sent in tokenized_sentences]

In [10]:
named_entities = []
for sentence in ne_annotated_sentences:
    temp_entity_name = ""
    temp_named_entity = None
    for term, tag in sentence:
        if tag !="O":
            temp_entity_name = " ".join([temp_entity_name, term]).strip()
            temp_named_entity = (temp_entity_name, tag)
        else:
            if temp_named_entity:
                named_entities.append(temp_named_entity)
                temp_entity_name = ""
                temp_named_entity = None
                
named_entities = list(set(named_entities))
entity_frame = pd.DataFrame(named_entities,
                           columns=["Entity Name", "Entity Type"])
print(entity_frame)

         Entity Name   Entity Type
0         Franz John        PERSON
1             Bayern  ORGANIZATION
2      Bayern Munich  ORGANIZATION
3            Germany      LOCATION
4  Franz Beckenbauer        PERSON
5          FC Bayern  ORGANIZATION
6             Munich      LOCATION
7            Bavaria      LOCATION


In [11]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = spacy.load("en_core_web_sm")

In [12]:
doc = nlp(text)
doc.ents

(Bayern Munich,
 FC Bayern,
 German,
 Munich,
 Bavaria,
 Germany,
 Bundesliga,
 German,
 German,
 26,
 18,
 FC Bayern,
 1900,
 eleven,
 Franz John,
 Bayern,
 1932,
 Bundesliga,
 1963,
 the middle of the 1970s,
 Franz Beckenbauer,
 the European Cup,
 three,
 1974-76,
 Bayern,
 ten,
 fifth,
 2013)

In [13]:
from pprint import pprint
pprint([(X.text, X.label_) for X in doc.ents])

[('Bayern Munich', 'ORG'),
 ('FC Bayern', 'ORG'),
 ('German', 'NORP'),
 ('Munich', 'GPE'),
 ('Bavaria', 'GPE'),
 ('Germany', 'GPE'),
 ('Bundesliga', 'GPE'),
 ('German', 'NORP'),
 ('German', 'NORP'),
 ('26', 'CARDINAL'),
 ('18', 'CARDINAL'),
 ('FC Bayern', 'NORP'),
 ('1900', 'DATE'),
 ('eleven', 'CARDINAL'),
 ('Franz John', 'PERSON'),
 ('Bayern', 'ORG'),
 ('1932', 'DATE'),
 ('Bundesliga', 'GPE'),
 ('1963', 'DATE'),
 ('the middle of the 1970s', 'DATE'),
 ('Franz Beckenbauer', 'PERSON'),
 ('the European Cup', 'EVENT'),
 ('three', 'CARDINAL'),
 ('1974-76', 'DATE'),
 ('Bayern', 'ORG'),
 ('ten', 'CARDINAL'),
 ('fifth', 'ORDINAL'),
 ('2013', 'DATE')]


In [14]:
len(doc.ents)

28

In [15]:
labels = [x.label_ for x in doc.ents]
Counter(labels)

Counter({'CARDINAL': 5,
         'DATE': 6,
         'EVENT': 1,
         'GPE': 5,
         'NORP': 4,
         'ORDINAL': 1,
         'ORG': 4,
         'PERSON': 2})

In [16]:
items = [x.text for x in doc.ents]
Counter(items).most_common(3)

[('German', 3), ('Bundesliga', 2), ('Bayern', 2)]

In [17]:
displacy.render(doc, jupyter=True, style='ent')

In [18]:
displacy.render(doc, style='dep', jupyter = True, options = {'distance': 100})

In [19]:
[(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in doc 
                                      if not y.is_stop and y.pos_ != 'PUNCT']]

[('\n', 'SPACE', '\n'),
 ('Bayern', 'PROPN', 'Bayern'),
 ('Munich', 'PROPN', 'Munich'),
 ('FC', 'PROPN', 'FC'),
 ('Bayern', 'PROPN', 'Bayern'),
 ('German', 'ADJ', 'german'),
 ('sports', 'NOUN', 'sport'),
 ('club', 'NOUN', 'club'),
 ('based', 'VERB', 'base'),
 ('Munich', 'PROPN', 'Munich'),
 ('\n', 'SPACE', '\n'),
 ('Bavaria', 'PROPN', 'Bavaria'),
 ('Germany', 'PROPN', 'Germany'),
 ('best', 'ADV', 'best'),
 ('known', 'VERB', 'know'),
 ('professional', 'ADJ', 'professional'),
 ('football', 'NOUN', 'football'),
 ('team', 'NOUN', 'team'),
 ('\n', 'SPACE', '\n'),
 ('plays', 'VERB', 'play'),
 ('Bundesliga', 'PROPN', 'Bundesliga'),
 ('tier', 'NOUN', 'tier'),
 ('German', 'ADJ', 'german'),
 ('football', 'NOUN', 'football'),
 ('\n', 'SPACE', '\n'),
 ('league', 'NOUN', 'league'),
 ('system', 'NOUN', 'system'),
 ('successful', 'ADJ', 'successful'),
 ('club', 'NOUN', 'club'),
 ('German', 'ADJ', 'german'),
 ('football', 'NOUN', 'football'),
 ('\n', 'SPACE', '\n'),
 ('history', 'NOUN', 'history'),
 (

In [20]:
dict([(str(x), x.label_) for x in doc.ents])

{'18': 'CARDINAL',
 '1900': 'DATE',
 '1932': 'DATE',
 '1963': 'DATE',
 '1974-76': 'DATE',
 '2013': 'DATE',
 '26': 'CARDINAL',
 'Bavaria': 'GPE',
 'Bayern': 'ORG',
 'Bayern Munich': 'ORG',
 'Bundesliga': 'GPE',
 'FC Bayern': 'NORP',
 'Franz Beckenbauer': 'PERSON',
 'Franz John': 'PERSON',
 'German': 'NORP',
 'Germany': 'GPE',
 'Munich': 'GPE',
 'eleven': 'CARDINAL',
 'fifth': 'ORDINAL',
 'ten': 'CARDINAL',
 'the European Cup': 'EVENT',
 'the middle of the 1970s': 'DATE',
 'three': 'CARDINAL'}